<a href="https://colab.research.google.com/github/nepalprabin/deeplearning-tensorflow/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install tensorflow-gpu==2.0.0 --ignore-installed

     |████████████████████████████████| 380.8MB 42kB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 71kB 11.9MB/s 
     |████████████████████████████████| 3.8MB 41.0MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 450kB 50.9MB/s 
     |████████████████████████████████| 20.2MB 1.2MB/s 
     |████████████████████████████████| 1.3MB 46.5MB/s 
     |████████████████████████████████| 2.9MB 43.1MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 112kB 62.8MB/s 
     |████████████████████████████████| 2.9MB 48.9MB/s 
     |████████████████████████████████| 92kB 14.2MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
     |████████████████████████████████| 583kB 57.6MB/s 
     |████████████████████████████████| 307kB 54.1MB/s 
     |████████████████████████████████| 163kB 57.1MB/s 
  

In [1]:
!pip install tqdm

In [2]:
#Downloading dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-04-13 12:35:48--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.217.208, 2607:f8b0:4007:801::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.217.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   321MB/s    in 0.2s    

2020-04-13 12:35:48 (321 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [5]:
#Data Preprocessing
# Importing dependencies
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0'

In [0]:
#Unzipping the dataset
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [0]:
zip_object.extractall('./')

In [0]:
zip_object.close()

In [0]:
#Setting up dataset paths
dataset_path_new = './cats_and_dogs_filtered/'

In [0]:
train_dir = os.path.join(dataset_path_new, 'train')
validation_dir = os.path.join(dataset_path_new, 'validation')

**Building the model**

In [0]:
#Loading the pre-trained model(MobileNetV2)
IMG_SHAPE = (128, 128, 3)

In [14]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 1s 0us/step


In [16]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [0]:
#Freezing the base model
base_model.trainable = False

In [18]:
#Defining the custom head for our network
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [20]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [0]:
#Defining the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [0]:
#Compiling the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics=['accuracy'])

In [26]:
#Creating Data Generators
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128, 128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [27]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128, 128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [28]:
#Training the model
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 18s 1s/step - loss: 0.7592 - accuracy: 0.5330 - val_loss: 0.6415 - val_accuracy: 0.6330
Epoch 2/5
16/16 [==============================] - 12s 734ms/step - loss: 0.6842 - accuracy: 0.6040 - val_loss: 0.5869 - val_accuracy: 0.6970
Epoch 3/5
16/16 [==============================] - 12s 732ms/step - loss: 0.6296 - accuracy: 0.6450 - val_loss: 0.5361 - val_accuracy: 0.7320
Epoch 4/5
16/16 [==============================] - 12s 734ms/step - loss: 0.5787 - accuracy: 0.6990 - val_loss: 0.4928 - val_accuracy: 0.7600
Epoch 5/5
16/16 [==============================] - 12s 730ms/step - loss: 0.5412 - accuracy: 0.7320 - val_loss: 0.4551 - val_accuracy: 0.7870


In [0]:
#Transfer learning model evaluation
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [33]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.7870000004768372


In [0]:
#Fine Tuning
#Unfreeze a few top top layers from the model
base_model.trainable = True

In [35]:
print("Number of layers in base model: {}".format(len(base_model.layers)))

Number of layers in base model: 155


In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [0]:
#Compiling the model for fine-tuning
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
#Fine tuning
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 14s 856ms/step - loss: 0.1926 - accuracy: 0.9260 - val_loss: 0.1009 - val_accuracy: 0.9650
Epoch 2/5
16/16 [==============================] - 13s 788ms/step - loss: 0.0300 - accuracy: 0.9975 - val_loss: 0.1122 - val_accuracy: 0.9620
Epoch 3/5
16/16 [==============================] - 13s 782ms/step - loss: 0.0104 - accuracy: 0.9990 - val_loss: 0.1164 - val_accuracy: 0.9640
Epoch 4/5
16/16 [==============================] - 12s 775ms/step - loss: 0.0065 - accuracy: 0.9985 - val_loss: 0.1609 - val_accuracy: 0.9550
Epoch 5/5
16/16 [==============================] - 12s 775ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.1890 - val_accuracy: 0.9570


In [0]:
#Evaluating the fine tuned model
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [41]:
print('Validation accuracy after fine tuning is: {}'.format(valid_accuracy))

Validation accuracy after fine tuning is: 0.9570000171661377
